# Computational Theory Assessment

**Student:** Tiffany Yong Ngik Chee  (G00425067)    
**Module:** Computation Theory  
**Lecturer:** Ian McLoughlin

This notebook contains solutions to five problems related to the [SHA-256 Secure Hash Standard (FIPS 180-4)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf).

---

## Problem 1: Binary Words and Operations

### Introduction

In this problem, I implement seven fundamental functions used in the SHA-256 cryptographic hash algorithm. These functions operate on 32-bit binary words and form the building blocks of the hash computation process.

All seven functions are defined in **Section 4.1.2** (pages 10-11) of the [Secure Hash Standard (FIPS 180-4)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf). They perform bitwise logical operations and rotations that help ensure the security and unpredictability of the SHA-256 hash function.

### Why 32-bit Operations?

SHA-256 processes data in 32-bit chunks (called "words"). Using numpy's `uint32` type ensures that all operations treat numbers as **unsigned 32-bit integers**, preventing overflow issues and ensuring compatibility with the standard's specifications.

In [7]:
# Import numpy for 32-bit unsigned integer operations.
# NumPy documentation: https://numpy.org/doc/stable/
import numpy as np

---

### Function 1: Parity(x, y, z)

#### What is the Parity Function?

The `Parity` function is defined in **Section 4.1.2, equation (4.3)** on page 10 of the standard. It is defined as:

$$\text{Parity}(x, y, z) = x \oplus y \oplus z$$

where $\oplus$ represents the bitwise XOR (exclusive OR) operation.

#### Why is it Used?

The Parity function is used in certain rounds of the SHA-256 compression function (specifically in the SHA-1 algorithm, which shares some operations with SHA-256). It provides [diffusion](https://en.wikipedia.org/wiki/Confusion_and_diffusion), meaning that changing a single bit in any of the inputs will affect the output, making the hash function more secure.

#### How Does XOR Work?

The XOR operation compares corresponding bits of two binary numbers:
- If the bits are **different**, the result is `1`
- If the bits are the **same**, the result is `0`

For three inputs, we XOR them sequentially: first `x ⊕ y`, then XOR that result with `z`.

In [8]:
def Parity(x, y, z):
    """
    Calculate the Parity function for SHA-256.
    
    As defined in Section 4.1.2 (equation 4.3) of FIPS 180-4,
    this function returns the bitwise XOR of three 32-bit words.
    
    The formula is: Parity(x, y, z) = x ⊕ y ⊕ z
    
    Parameters
    ----------
    x : int or numpy.uint32
        First 32-bit word
    y : int or numpy.uint32
        Second 32-bit word
    z : int or numpy.uint32
        Third 32-bit word
        
    Returns
    -------
    numpy.uint32
        The bitwise XOR of x, y, and z
        
    References
    ----------
    FIPS 180-4, Section 4.1.2, page 10
    https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
    """
    # Ensure all inputs are treated as 32-bit unsigned integers.
    # This prevents overflow and ensures compatibility with the standard.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Perform bitwise XOR operation.
    # The ^ operator in Python performs bitwise XOR.
    # See: https://docs.python.org/3/reference/expressions.html#binary-bitwise-operations
    return x ^ y ^ z

#### Understanding the XOR Operation

Here is the demonstrate how XOR works with a simple example using smaller numbers for clarity:

In [9]:
# Example with small numbers to show how XOR works.
x_example = 0b1100  # Binary: 1100 (decimal: 12)
y_example = 0b1010  # Binary: 1010 (decimal: 10)
z_example = 0b1111  # Binary: 1111 (decimal: 15)

print("Example XOR operation:")
print(f"x = {x_example:04b} ({x_example})")
print(f"y = {y_example:04b} ({y_example})")
print(f"z = {z_example:04b} ({z_example})")
print(f"x ⊕ y = {x_example ^ y_example:04b} ({x_example ^ y_example})")
print(f"(x ⊕ y) ⊕ z = {x_example ^ y_example ^ z_example:04b} ({x_example ^ y_example ^ z_example})")

Example XOR operation:
x = 1100 (12)
y = 1010 (10)
z = 1111 (15)
x ⊕ y = 0110 (6)
(x ⊕ y) ⊕ z = 1001 (9)


#### Testing the Parity Function

Now let's test the `Parity` function with actual 32-bit values as used in SHA-256:

In [10]:
# Test the Parity function with 32-bit hexadecimal values.
# Using hexadecimal notation (0x) as it's standard in cryptography.
x_test = 0x12345678
y_test = 0xABCDEF00
z_test = 0xFFFFFFFF

result = Parity(x_test, y_test, z_test)

print("Testing Parity Function:")
print(f"x = 0x{x_test:08x}")
print(f"y = 0x{y_test:08x}")
print(f"z = 0x{z_test:08x}")
print(f"Parity(x, y, z) = 0x{result:08x}")
print(f"Result type: {type(result)}")

Testing Parity Function:
x = 0x12345678
y = 0xabcdef00
z = 0xffffffff
Parity(x, y, z) = 0x46064687
Result type: <class 'numpy.uint32'>


#### My Understanding: Why Parity Works

After studying the Parity function, I understand that:

1. **XOR is associative**: `(a ⊕ b) ⊕ c = a ⊕ (b ⊕ c)`, so the order doesn't matter
2. **XOR is self-inverse**: `a ⊕ a = 0`, which makes it useful for cryptography
3. **Bit independence**: Each bit position is processed independently

Let me verify this with my own test case:

In [11]:
# My own test: Verify XOR properties
# Test 1: XOR with itself should give 0
a = np.uint32(0x12345678)
print(f"Test 1 - Self-inverse property:")
print(f"{a:08x} ⊕ {a:08x} = {a ^ a:08x} (should be 0)")
print()

# Test 2: XOR with 0 should give the original value
print(f"Test 2 - Identity property:")
print(f"{a:08x} ⊕ 00000000 = {a ^ np.uint32(0):08x} (should be {a:08x})")
print()

# Test 3: Associativity - order doesn't matter
x = np.uint32(0xAAAAAAAA)
y = np.uint32(0x55555555)
z = np.uint32(0xF0F0F0F0)
method1 = Parity(x, y, z)
method2 = (x ^ y) ^ z
method3 = x ^ (y ^ z)
print(f"Test 3 - Associativity:")
print(f"Parity(x,y,z) = {method1:08x}")
print(f"(x ⊕ y) ⊕ z   = {method2:08x}")
print(f"x ⊕ (y ⊕ z)   = {method3:08x}")
print(f"All equal: {method1 == method2 == method3}")

Test 1 - Self-inverse property:
12345678 ⊕ 12345678 = 00000000 (should be 0)

Test 2 - Identity property:
12345678 ⊕ 00000000 = 12345678 (should be 12345678)

Test 3 - Associativity:
Parity(x,y,z) = 0f0f0f0f
(x ⊕ y) ⊕ z   = 0f0f0f0f
x ⊕ (y ⊕ z)   = 0f0f0f0f
All equal: True


#### Verification

The test confirms that:
1. The function returns a `numpy.uint32` type, ensuring 32-bit operations
2. The XOR operation works correctly on full 32-bit words
3. The result is displayed in hexadecimal format, which is standard in cryptographic contexts

---


### Function 2: Ch(x, y, z)

#### What is the Ch Function?

The `Ch` function (short for "Choose") is defined in **Section 4.1.2, equation (4.2)** on page 10 of the standard:

$$\text{Ch}(x, y, z) = (x \land y) \oplus (\neg x \land z)$$

where:
- $\land$ represents bitwise AND
- $\oplus$ represents bitwise XOR
- $\neg$ represents bitwise NOT (complement)

#### Why is it Called "Choose"?

The Ch function is called "choose" because it uses `x` as a **selector**:
- When a bit in `x` is **1**, the corresponding bit from `y` is chosen
- When a bit in `x` is **0**, the corresponding bit from `z` is chosen

This can also be written as: **"x chooses between y and z"**

#### Logical Explanation

The formula works as follows:
1. `(x & y)` - Where x has 1's, keep the bits from y
2. `(~x & z)` - Where x has 0's (meaning ~x has 1's), keep the bits from z
3. XOR these together to get the final result

This function provides [confusion](https://en.wikipedia.org/wiki/Confusion_and_diffusion) in the cryptographic sense, making the relationship between the key and ciphertext complex.

# Problem 2: Fractional Parts of Cube Roots

# Problem 3: Padding

# Problem 4: Hashes

# Problem 5: Passwords

# End